# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Cities_df=pd.read_csv("output_data/cities.csv")
Cities_df.head(5)



,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,vaini,-21.2000,-175.2000,82.40,89,75,9.22,TO,1612580738
1,1,rikitea,-23.1203,-134.9692,77.67,75,5,18.99,PF,1612580739
2,2,mahebourg,-20.4081,57.7000,78.80,83,75,10.36,MU,1612580562
3,3,saldanha,-33.0117,17.9442,64.42,92,0,15.70,ZA,1612580740
4,4,poum,-20.2333,164.0167,80.01,70,6,7.09,NC,1612580740


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
locations = Cities_df[["Lat", "Lng"]].astype(float)
weights_=Cities_df['Humidity'].astype(float)

In [9]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(45,-5),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights_, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = Cities_df[(Cities_df['Max Temp']>70)
                          &(Cities_df['Wind Speed']
                          <8)&(Cities_df['Max Temp']<95)&(Cities_df['Humidity']>40)
                          &(Cities_df['Humidity']<75)&(Cities_df['Cloudiness']<10)]
ideal_weather.count()

KeyError: 'Max Temp'

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df=ideal_weather
header_list=['City','Hotel Name','Country','Lat','Lng']
hotel_df = hotel_df.reindex(columns = header_list)
hotel_df.head(1)


,City,Hotel Name,Country,Lat,Lng
70,ponta do sol,,BR,-20.63,-46.0


In [39]:
#url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": f'{hotel_df["Lat"],hotel_df["Lng"]}',
    "radius": 5000,
    "keyword":"hotel",
    "type": "hotel",
    "key": g_key,
}


# use iterrows to iterate through pandas dataframe
for index,row in hotel_df.iterrows():

    # get restaurant type from df
    City = row['City']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {City} is {results[0]['name']}.")
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
print("-------End-------")

Retrieving Results for Index 70: ponta do sol.
Missing field/result... skipping.
------------
Retrieving Results for Index 88: cidreira.
Missing field/result... skipping.
------------
Retrieving Results for Index 297: mogok.
Missing field/result... skipping.
------------
Retrieving Results for Index 494: baylo.
Missing field/result... skipping.
------------
Retrieving Results for Index 512: natchez.
Missing field/result... skipping.
------------
Retrieving Results for Index 518: bonito.
Missing field/result... skipping.
------------
Retrieving Results for Index 525: mecca.
Missing field/result... skipping.
------------
Retrieving Results for Index 527: tomatlan.
Missing field/result... skipping.
------------
-------End-------


In [40]:
hotel_df

,City,Hotel Name,Country,Lat,Lng
70,ponta do sol,,BR,-20.63,-46.00
88,cidreira,,BR,-30.17,-50.22
297,mogok,,MM,22.92,96.51
494,baylo,,PH,7.85,126.06
512,natchez,,US,31.56,-91.40
518,bonito,,BR,-21.13,-56.49
525,mecca,,SA,21.43,39.83
527,tomatlan,,MX,19.93,-105.25


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
